In [6]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE
from pipelines.tasks.config.config_irisge import get_irisge_config
import geopandas as gpd
from pipelines.tasks.config.common import CACHE_FOLDER
from pathlib import Path

irisge_conf = get_irisge_config()

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)
tables = con.execute("SHOW TABLES").fetchall()
# show all tables in db
for table in tables:
    print(tables)
# con.close()

[('cog_communes',), ('irig_ge',), ('irig_ge_test',)]
[('cog_communes',), ('irig_ge',), ('irig_ge_test',)]
[('cog_communes',), ('irig_ge',), ('irig_ge_test',)]


In [2]:
# show irisge table colummes info
query = f"DESCRIBE {irisge_conf['file']['table_name']}"
column_info = con.execute(query).fetchall()

for col in column_info:
    print(col)

('ID', 'VARCHAR', 'YES', None, None, None)
('INSEE_COM', 'VARCHAR', 'YES', None, None, None)
('NOM_COM', 'VARCHAR', 'YES', None, None, None)
('IRIS', 'VARCHAR', 'YES', None, None, None)
('CODE_IRIS', 'VARCHAR', 'YES', None, None, None)
('NOM_IRIS', 'VARCHAR', 'YES', None, None, None)
('TYP_IRIS', 'VARCHAR', 'YES', None, None, None)
('GEOM', 'VARCHAR', 'YES', None, None, None)


In [ ]:
query = f"SELECT INSEE_COM, GEOM FROM {irisge_conf['file']['table_name']}"
iris_ge = con.execute(query)

top5 = iris_ge.df().head(5)
print(top5)
#

  INSEE_COM                                               GEOM
0     67043  POLYGON ((1050772.9 6847169.1, 1050902.2 68472...
1     13202  MULTIPOLYGON (((889986 6251412.3, 890200.1 625...
2     56185  POLYGON ((216571.6 6760628.4, 216572.7 6760636...
3     93063  POLYGON ((657944.3 6865938.3, 657957.7 6866026...
4     94048  POLYGON ((666452.1 6849977.7, 666590.8 6850199...


In [ ]:
# read duckdb and convert the data into geoJson
query = f"SELECT  * FROM {irisge_conf['file']['table_name']}"
iris_df = con.execute(query).fetchdf()
gdf = gpd.GeoDataFrame(iris_df, geometry=gpd.GeoSeries.from_wkt(iris_df["wkt"]))

geo_json = "output_test.geojson"
geo_json_path = Path(CACHE_FOLDER, geo_json)

gdf.to_file(geo_json_path, driver="GeoJSON")
# print(iris_df.head())
print("geojson generated successfully")

/Users/llf/open-source/13_pollution_eau_d4g/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


geojson generated successfully
